In [1]:
import pandas as pd
import re

def parse_condition(condition):
    """Extrai coluna, operador e valor de uma condição."""
    condition = condition.strip()

    match_grupo = re.match(r"\[GRUPO DE DESPESA\]\.\[Código\]\s*=\s*'(.*?)'", condition)
    if match_grupo:
        return 'GRUPO DE DESPESA', '=', match_grupo.group(1)

    match_unidade = re.match(r"\[UNIDADE ORÇAMENTÁRIA\]\.\[Código\]\s*=\s*'(.*?)'", condition)
    if match_unidade:
        return 'UNIDADE ORÇAMENTÁRIA', '=', match_unidade.group(1)

    match_acao_termina = re.match(r"\[AÇÃO PPA\]\.\[Código\] TERMINA COM '(.*?)'", condition)
    if match_acao_termina:
        return 'AÇÃO PPA', 'TERMINA COM', match_acao_termina.group(1)

    print(f"Condição não reconhecida: '{condition}'")
    return None, None, None


In [2]:
# 

# LME 1

In [3]:
# Nome do seu arquivo de texto
lme_1 = 'LME_1_antes.txt'

with open(lme_1, 'r', encoding='utf-8') as f:
    conteudo = f.read()

# Dividir o conteúdo por " OU " e remover parênteses e espaços extras
grupos_condicoes = [grupo.strip()[1:-1].strip() for grupo in conteudo.split(' OU ')]

data = []
for grupo in grupos_condicoes:
    condicoes = [c.strip() for c in grupo.split(' E ')]
    grupo_data = {}
    for condicao in condicoes:
        coluna, operador, valor = parse_condition(condicao)
        if coluna:
            chave = f'{coluna} ({operador})'
            grupo_data[chave] = valor
    if grupo_data:
        data.append(grupo_data)

# Criar o DataFrame
df_lme1_antes = pd.DataFrame(data)
df_lme1_antes['chave'] = df_lme1_antes['GRUPO DE DESPESA (=)'] + df_lme1_antes['UNIDADE ORÇAMENTÁRIA (=)'] + df_lme1_antes['AÇÃO PPA (TERMINA COM)']
df_lme1_antes

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave
0,1,40440,2019,1404402019
1,3,40440,2019,3404402019
2,1,40440,2022,1404402022
3,3,40440,2022,3404402022
4,1,37020,2024,1370202024
...,...,...,...,...
265,3,20342,8056,3203428056
266,1,20610,8178,1206108178
267,3,20610,8178,3206108178
268,1,20342,8179,1203428179


In [4]:
# Nome do seu arquivo de texto
lme_1 = 'LME_1_depois.txt'

with open(lme_1, 'r', encoding='utf-8') as f:
    conteudo = f.read()

# Dividir o conteúdo por " OU " e remover parênteses e espaços extras
grupos_condicoes = [grupo.strip()[1:-1].strip() for grupo in conteudo.split(' OU ')]

data = []
for grupo in grupos_condicoes:
    condicoes = [c.strip() for c in grupo.split(' E ')]
    grupo_data = {}
    for condicao in condicoes:
        coluna, operador, valor = parse_condition(condicao)
        if coluna:
            chave = f'{coluna} ({operador})'
            grupo_data[chave] = valor
    if grupo_data:
        data.append(grupo_data)

# Criar o DataFrame
df_lme1_depois = pd.DataFrame(data)
df_lme1_depois['chave'] = df_lme1_depois['GRUPO DE DESPESA (=)'] + df_lme1_depois['UNIDADE ORÇAMENTÁRIA (=)'] + df_lme1_depois['AÇÃO PPA (TERMINA COM)']
df_lme1_depois

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave
0,1,40440,2019,1404402019
1,3,40440,2019,3404402019
2,1,40440,2022,1404402022
3,3,40440,2022,3404402022
4,1,37020,2024,1370202024
...,...,...,...,...
274,1,27530,2175,1275302175
275,1,27530,2660,1275302660
276,3,27530,2660,3275302660
277,1,27530,4637,1275304637


In [5]:
lme_1_inner = pd.merge(df_lme1_antes, df_lme1_depois, how='inner', on='chave')
lme_1_outer = pd.merge(df_lme1_antes, df_lme1_depois, how='outer', on='chave')

concat_lme1 = pd.concat([df_lme1_antes, df_lme1_depois])
concat_lme1_cont = concat_lme1['chave'].value_counts().reset_index()
concat_lme1_cont_2 = concat_lme1_cont.query('count == 2')
concat_lme1_cont_1 = concat_lme1_cont.query('count == 1')
lme_1_count = pd.merge(concat_lme1_cont_1, lme_1_outer, how='left', on='chave')
lme_entrou = lme_1_count['UNIDADE ORÇAMENTÁRIA (=)_y'].value_counts().sum()
lme_saiu = lme_1_count['UNIDADE ORÇAMENTÁRIA (=)_x'].value_counts().sum()
print(f'No LME 1 tivmeos {lme_saiu} que saíram e {lme_entrou} que entraram na regra')
lme_1_count

No LME 1 tivmeos 2 que saíram e 11 que entraram na regra


,chave,count,GRUPO DE DESPESA (=)_x,UNIDADE ORÇAMENTÁRIA (=)_x,AÇÃO PPA (TERMINA COM)_x,GRUPO DE DESPESA (=)_y,UNIDADE ORÇAMENTÁRIA (=)_y,AÇÃO PPA (TERMINA COM)_y
0,1274105679,1,NaN,NaN,NaN,1,27410,5679
1,1275304637,1,NaN,NaN,NaN,1,27530,4637
2,3275302660,1,NaN,NaN,NaN,3,27530,2660
3,1275302660,1,NaN,NaN,NaN,1,27530,2660
4,1275302175,1,NaN,NaN,NaN,1,27530,2175
5,1274108184,1,NaN,NaN,NaN,1,27410,8184
6,3274102660,1,NaN,NaN,NaN,3,27410,2660
7,1274102840,1,NaN,NaN,NaN,1,27410,2840
8,1274102839,1,NaN,NaN,NaN,1,27410,2839
9,1274102660,1,NaN,NaN,NaN,1,27410,2660


# LME 2

In [6]:
# Nome do seu arquivo de texto
lme_2 = 'LME_2_antes.txt'

with open(lme_2, 'r', encoding='utf-8') as f:
    conteudo = f.read()

# Dividir o conteúdo por " OU " e remover parênteses e espaços extras
grupos_condicoes = [grupo.strip()[1:-1].strip() for grupo in conteudo.split(' OU ')]

data = []
for grupo in grupos_condicoes:
    condicoes = [c.strip() for c in grupo.split(' E ')]
    grupo_data = {}
    for condicao in condicoes:
        coluna, operador, valor = parse_condition(condicao)
        if coluna:
            chave = f'{coluna} ({operador})'
            grupo_data[chave] = valor
    if grupo_data:
        data.append(grupo_data)

# Criar o DataFrame
df_lme2_antes = pd.DataFrame(data)
df_lme2_antes['chave'] = df_lme2_antes['GRUPO DE DESPESA (=)'] + df_lme2_antes['UNIDADE ORÇAMENTÁRIA (=)'] + df_lme2_antes['AÇÃO PPA (TERMINA COM)']
df_lme2_antes

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave
0,1,57010,0002,1570100002
1,2,57010,0002,2570100002
2,3,57010,0002,3570100002
3,4,57010,0002,4570100002
4,5,57010,0002,5570100002
...,...,...,...,...
1029,3,20342,9997,3203429997
1030,4,20342,9997,4203429997
1031,5,20342,9997,5203429997
1032,6,20342,9997,6203429997


In [17]:
df_lme2_antes.query('`UNIDADE ORÇAMENTÁRIA (=)` == "27410" and `GRUPO DE DESPESA (=)` == "3" and `AÇÃO PPA (TERMINA COM)` == "2010"')

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave


In [7]:
# Nome do seu arquivo de texto
lme_2 = 'LME_2_depois.txt'

with open(lme_2, 'r', encoding='utf-8') as f:
    conteudo = f.read()

# Dividir o conteúdo por " OU " e remover parênteses e espaços extras
grupos_condicoes = [grupo.strip()[1:-1].strip() for grupo in conteudo.split(' OU ')]

data = []
for grupo in grupos_condicoes:
    condicoes = [c.strip() for c in grupo.split(' E ')]
    grupo_data = {}
    for condicao in condicoes:
        coluna, operador, valor = parse_condition(condicao)
        if coluna:
            chave = f'{coluna} ({operador})'
            grupo_data[chave] = valor
    if grupo_data:
        data.append(grupo_data)

# Criar o DataFrame
df_lme2_depois = pd.DataFrame(data)
df_lme2_depois['chave'] = df_lme2_depois['GRUPO DE DESPESA (=)'] + df_lme2_depois['UNIDADE ORÇAMENTÁRIA (=)'] + df_lme2_depois['AÇÃO PPA (TERMINA COM)']
df_lme2_depois

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave
0,1,57010,0002,1570100002
1,2,57010,0002,2570100002
2,3,57010,0002,3570100002
3,4,57010,0002,4570100002
4,5,57010,0002,5570100002
...,...,...,...,...
1071,2,27530,2010,2275302010
1072,3,27530,2010,3275302010
1073,4,27530,2010,4275302010
1074,5,27530,2010,5275302010


In [13]:
df_lme2_depois.query('`UNIDADE ORÇAMENTÁRIA (=)` == "27410" and `GRUPO DE DESPESA (=)` == "3" and `AÇÃO PPA (TERMINA COM)` == "2010"')

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave
1054,3,27410,2010,3274102010


In [18]:
# Nome do seu arquivo de texto
lme_2 = 'LME_2_atual.txt'

with open(lme_2, 'r', encoding='utf-8') as f:
    conteudo = f.read()

# Dividir o conteúdo por " OU " e remover parênteses e espaços extras
grupos_condicoes = [grupo.strip()[1:-1].strip() for grupo in conteudo.split(' OU ')]

data = []
for grupo in grupos_condicoes:
    condicoes = [c.strip() for c in grupo.split(' E ')]
    grupo_data = {}
    for condicao in condicoes:
        coluna, operador, valor = parse_condition(condicao)
        if coluna:
            chave = f'{coluna} ({operador})'
            grupo_data[chave] = valor
    if grupo_data:
        data.append(grupo_data)

# Criar o DataFrame
df_lme2_atual = pd.DataFrame(data)
df_lme2_atual['chave'] = df_lme2_atual['GRUPO DE DESPESA (=)'] + df_lme2_atual['UNIDADE ORÇAMENTÁRIA (=)'] + df_lme2_atual['AÇÃO PPA (TERMINA COM)']
df_lme2_atual

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave
0,1,57010,0002,1570100002
1,2,57010,0002,2570100002
2,3,57010,0002,3570100002
3,4,57010,0002,4570100002
4,5,57010,0002,5570100002
...,...,...,...,...
1053,2,27530,0467,2275300467
1054,3,27530,0467,3275300467
1055,4,27530,0467,4275300467
1056,5,27530,0467,5275300467


In [19]:
df_lme2_atual.query('`UNIDADE ORÇAMENTÁRIA (=)` == "27410" and `GRUPO DE DESPESA (=)` == "3" and `AÇÃO PPA (TERMINA COM)` == "2010"')

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave


In [8]:
lme_2_inner = pd.merge(df_lme2_antes, df_lme2_depois, how='inner', on='chave')
lme_2_outer = pd.merge(df_lme2_antes, df_lme2_depois, how='outer', on='chave')

concat_lme2= pd.concat([df_lme2_antes, df_lme2_depois])
concat_lme2_cont = concat_lme2['chave'].value_counts().reset_index()
concat_lme2_cont_2 = concat_lme2_cont.query('count == 2')
concat_lme2_cont_1 = concat_lme2_cont.query('count == 1')
lme_2_count = pd.merge(concat_lme2_cont_1, lme_2_outer, how='left', on='chave')
lme_entrou = lme_2_count['UNIDADE ORÇAMENTÁRIA (=)_y'].value_counts().sum()
lme_saiu = lme_2_count['UNIDADE ORÇAMENTÁRIA (=)_x'].value_counts().sum()
print(f'No LME 2 tivmeos {lme_saiu} que saíram e {lme_entrou} que entraram na regra')
lme_2_count

No LME 2 tivmeos 0 que saíram e 42 que entraram na regra


,chave,count,GRUPO DE DESPESA (=)_x,UNIDADE ORÇAMENTÁRIA (=)_x,AÇÃO PPA (TERMINA COM)_x,GRUPO DE DESPESA (=)_y,UNIDADE ORÇAMENTÁRIA (=)_y,AÇÃO PPA (TERMINA COM)_y
0,6274102010,1,NaN,NaN,NaN,6,27410,2010
1,2275300467,1,NaN,NaN,NaN,2,27530,0467
2,1275300016,1,NaN,NaN,NaN,1,27530,0016
3,2275300016,1,NaN,NaN,NaN,2,27530,0016
4,3275300016,1,NaN,NaN,NaN,3,27530,0016
5,4275300016,1,NaN,NaN,NaN,4,27530,0016
6,5275300016,1,NaN,NaN,NaN,5,27530,0016
7,6275300016,1,NaN,NaN,NaN,6,27530,0016
8,1275300467,1,NaN,NaN,NaN,1,27530,0467
9,2275302010,1,NaN,NaN,NaN,2,27530,2010


# LME 6

In [9]:
# Nome do seu arquivo de texto
lme_6 = 'LME_6_antes.txt'

with open(lme_6, 'r', encoding='utf-8') as f:
    conteudo = f.read()

# Dividir o conteúdo por " OU " e remover parênteses e espaços extras
grupos_condicoes = [grupo.strip()[1:-1].strip() for grupo in conteudo.split(' OU ')]

data = []
for grupo in grupos_condicoes:
    condicoes = [c.strip() for c in grupo.split(' E ')]
    grupo_data = {}
    for condicao in condicoes:
        coluna, operador, valor = parse_condition(condicao)
        if coluna:
            chave = f'{coluna} ({operador})'
            grupo_data[chave] = valor
    if grupo_data:
        data.append(grupo_data)

# Criar o DataFrame
df_lme6_antes = pd.DataFrame(data)
df_lme6_antes['chave'] = df_lme6_antes['GRUPO DE DESPESA (=)'] + df_lme6_antes['UNIDADE ORÇAMENTÁRIA (=)'] + df_lme6_antes['AÇÃO PPA (TERMINA COM)']
df_lme6_antes

,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave
0,3,18010,2299,3180102299
1,3,52010,4596,3520104596
2,3,25010,4597,3250104597
3,3,54010,8021,3540108021
4,3,14759,8021,3147598021
...,...,...,...,...
89,3,31330,8021,3313308021
90,3,31710,8021,3317108021
91,3,30010,8021,3300108021
92,3,53010,8021,3530108021


In [10]:
# Nome do seu arquivo de texto
lme_6 = 'LME_6_depois.txt'

with open(lme_6, 'r', encoding='utf-8') as f:
    conteudo = f.read()

# Dividir o conteúdo por " OU " e remover parênteses e espaços extras
grupos_condicoes = [grupo.strip()[1:-1].strip() for grupo in conteudo.split(' OU ')]

data = []
for grupo in grupos_condicoes:
    condicoes = [c.strip() for c in grupo.split(' E ')]
    grupo_data = {}
    for condicao in condicoes:
        coluna, operador, valor = parse_condition(condicao)
        if coluna:
            chave = f'{coluna} ({operador})'
            grupo_data[chave] = valor
    if grupo_data:
        data.append(grupo_data)

# Criar o DataFrame
df_lme6_depois = pd.DataFrame(data)
df_lme6_depois['chave'] = df_lme6_depois['GRUPO DE DESPESA (=)'] + df_lme6_depois['UNIDADE ORÇAMENTÁRIA (=)'] + df_lme6_depois['AÇÃO PPA (TERMINA COM)']
df_lme6_depois


,GRUPO DE DESPESA (=),UNIDADE ORÇAMENTÁRIA (=),AÇÃO PPA (TERMINA COM),chave
0,3,18010,2299,3180102299
1,3,52010,4596,3520104596
2,3,25010,4597,3250104597
3,3,54010,8021,3540108021
4,3,61010,8021,3610108021
...,...,...,...,...
91,3,53410,8021,3534108021
92,3,53010,8021,3530108021
93,3,65010,8021,3650108021
94,3,27410,8021,3274108021


In [11]:
lme_6_inner = pd.merge(df_lme6_antes, df_lme6_depois, how='inner', on='chave')
lme_6_outer = pd.merge(df_lme6_antes, df_lme6_depois, how='outer', on='chave')

concat_lme6 = pd.concat([df_lme6_antes, df_lme6_depois])
concat_lme6_cont = concat_lme6['chave'].value_counts().reset_index()
concat_lme6_cont_2 = concat_lme6_cont.query('count == 2')
concat_lme6_cont_1 = concat_lme6_cont.query('count == 1')
lme_6_count = pd.merge(concat_lme6_cont_1, lme_6_outer, how='left', on='chave')
lme_entrou = lme_6_count['UNIDADE ORÇAMENTÁRIA (=)_y'].value_counts().sum()
lme_saiu = lme_6_count['UNIDADE ORÇAMENTÁRIA (=)_x'].value_counts().sum()
print(f'No LME 6 tivmeos {lme_saiu} que saíram e {lme_entrou} que entraram na regra')
lme_6_count

No LME 6 tivmeos 0 que saíram e 2 que entraram na regra


,chave,count,GRUPO DE DESPESA (=)_x,UNIDADE ORÇAMENTÁRIA (=)_x,AÇÃO PPA (TERMINA COM)_x,GRUPO DE DESPESA (=)_y,UNIDADE ORÇAMENTÁRIA (=)_y,AÇÃO PPA (TERMINA COM)_y
0,3274108021,1,NaN,NaN,NaN,3,27410,8021
1,3275308021,1,NaN,NaN,NaN,3,27530,8021
